In [1]:
from math import log10


In [2]:
from ctypes import c_uint, c_float

def cast(num, rettype):
    """
    Given a ctypes type, it attempts to read read a number from 
    some memory type and interpret it as a different type
    This will have overhead, but will provide a cast like interface
    """
    return rettype.from_buffer(num).value


def cast_to_float(n):
    return cast(c_uint(n), c_float)

def cast_to_int(n):
    return cast(c_float(n), c_uint)




In [7]:
def fast_inv_sqrt(num):
    halfnum = 0.5 * num
    # take the number, make it a float and read it into an integer
    inum = cast_to_int(num)
    # use the hack
    hack = 0x5f3759df - (inum >> 1)
    # back into floating point
    fnum = cast_to_float(hack)
    print(f"guess {fnum}")
    return fnum * (1.5 - halfnum * fnum * fnum) 
    
def inv_sqrt(num):
    return 1/num**(0.5)

In [8]:
test_num = 250
(fast_inv_sqrt(test_num) - inv_sqrt(test_num))**2

guess 0.06112086400389671


1.1207861519762848e-08

# Floating Point Numbers

For the sake of simplicity, let us look at 32 bit floats. A 32 bit float is divided into 3 sections:

1. The first bit for the sign. 1 in this location represents a negative number
2. The next 8 bits for the exponent. This uses unsigned representation, but is normalised by subtracting -127 to get the floating point ranges. Note, this is not using 2's complement for it's representation. 1 in this space represents -126 and 0x80 represents 1 for example. (It's effectively a negative 2's complement)
3. The remaining 23 bits for the mantissa. This is represented in fixed point with 1 to be added. I.e. the leading bit represents 0.5 to which 1 is added for the mantissa expression. 

So a floating point is given by by
`sign * (1 + mantissa) * 2 ^ exponent`

There is a special denormalised mode, when the exponent is 0, the exponent is treated as -126 and the 1+mantissa becomes just mantissa.

This link here: https://www.h-schmidt.net/FloatConverter/IEEE754.html should give you some additional insight. And Wikipedia: https://en.wikipedia.org/wiki/Single-precision_floating-point_format explains it in depth too

Some of this should explain things we've seen using floating point

1. Since the mantissa is fixed point, the (1+mantissa) is effectively from 1 to 2
2. That the largest number is 2 * 2 ^ 127 = 2^ 128 ~ 3.4 * 10 ^ 38
3. Smallest positive is obtained from the denormalised mode with is 2^-126 * 2 ^-23 = 2 ^ -149 ~ 7.0 x 10^-46
4. Smallest change in an arbitrary floating point is given by 2 ^ -23 ~ 1.19 * 10 ^-7

In [186]:
def float_report(num):
    # convert number to float and read in as an int
    try:
        num = cast_to_int(num)
    except TypeError:
        # we were passed a string. 
        assert num[:2] == "0x"
        num = int(num, 16)
    # convert to binary and drop the leading 0b and pad to 32 bits
    num = bin(num)[2:].zfill(32)
    sign = int(num[0])
    exponent = int(num[1:9], 2)
    mantissa = int(num[9:], 2)
    # report mantissa as hex
    print(f"Sign: {sign}, exp: {exponent}, mant: {hex(mantissa)}")
    return sign, exponent, mantissa

def make_float(sign, exponent, mantissa):
    mant = bin(mantissa)[2:].zfill(23)
    
    exp = bin(exponent)[2:].zfill(8)
    # check we've got sensible numbers
    assert len(mant) == 23
    assert len(exp) == 8
    assert sign in (0,1)
    sign = str(sign)
    
    bitsrep = sign + exp + mant
    bint = int(bitsrep, 2)
    #print(f"Bin rep : {bitsrep}")
    print(f"Hex Rep : {hex(bint).zfill(8)}")
    return cast_to_float(bint)

In [187]:
float_report(-128.0)
make_float(1, 134, 0)

Sign: 1, exp: 134, mant: 0x0
Hex Rep : 0xc3000000


-128.0

# Fast Inverse Square Root

The Fast inverse square root is just Newton's method with a good initial guess. Follow the maths here for some more information: https://betterexplained.com/articles/understanding-quakes-fast-inverse-square-root/. 


# The Initial guess

This line`0x5f3759df - (inum >> 1)` is the secret sauce above, getting a good approximation. Let's break this down.

Noting that the exponent is a bit out of sync with the mantissa, let's rewrite the magic number slightly. This can be seen by doing a left bitshift on the two leading bits and realizing thar leading 3 in the mantissa means the MSB is 0

1. Exponent = 0xbe = 190
2. Mantissa = 0x3759df

In [178]:
hex(0x5f << 1)

'0xbe'

In [146]:
0xbe

190

In [148]:
float_report("0x5f3759df")
hex(3627487)

Sign: 0, exp: 190, mant: 3627487


'0x3759df'

## The exponent

Let's simplify this to a problem which is purely a power of 2. Let's consider `2^4` and how we would use bit manipulation to get `2^-2` as the answer. We're going to deal with odd powers later (since we need to manipulate the mantissa too)


Noting that we want a good estimate of the inverse square root, when provided a number in exponent form, we instantly know that halving and inverting the exponent (i.e. making 10 ^ 4 into 10 ^ -2) is a good idea. So that's what we do.

For a simple unsigned integer, doing a right shift is enough to halve it's value, i.e. n >> 1. Again, remembering that the exponent is an unsigned 8 bit integer with a -127 offset. To halve a value stored in the exponent form we need to do  
`exponent/2 = (exponent - 127) >> 1  + 127`

And using this logic, to get the negative half  
`-exponent/2 = 127 - ((exponent - 127) >> 1)`

Expanding the above we get

`127 - (exponent -126 -1) >> 1`  
`127 + 63 - (exponent -1) >> 1`  
`190 - (exponent -1) >> 1`  

Depending on whether we have an odd or even exponent we need to decide on the result of the above. Note even exponents, such as `2^4` would have the float exponent set to `131`. So the above simplifies to 

`190 - (exponent >> 1)`

For even exponents


Note that 190 is `0xbe` and we represent this mask as `0x5f000000` after we shift it to the right to make space for the sign bit. 


In [167]:
hex(0xbe0 >> 1)

'0x5f0'

In [149]:
float_report(64)

Sign: 0, exp: 133, mant: 0


(0, 133, 0)

In [150]:
float_report(1/8)

Sign: 0, exp: 124, mant: 0


(0, 124, 0)

In [169]:
float_report("0x5f000000")

Sign: 0, exp: 190, mant: 0


(0, 190, 0)

In [190]:
def inv_sqrt(num):
    im = 0x5f000000 - (cast_to_int(num) >> 1)
    return cast_to_float(im)

In [191]:
inv_sqrt(64)

0.09375

In [192]:
inv_sqrt(32)

0.125

In [193]:
1/8 

0.125

We're not quite there, what happened? Well, when we have floating point numbers, our shift from the exponent falls into the mantissa. This makes our mantissa from 1 to 1.5. When we have an odd exponent, we don't have this scale factor happen and instead get ceiling of the exponent halved. 
Hence 2^5 = 32 will have 2^-3 at the end of the above process, while 2^6  will have a value shifted into the mantissa that results the exponent getting a value of -3 and the subtraction results in another drop for the exponent to -4 which is then boosted by the mantissa going from 1 to 1.5.
To fix this, we're going to bitmask the result of the division so we have the expected division each time, i.e. we take the ceiling of the power 


In [194]:
float_report(64)

Sign: 0, exp: 133, mant: 0x0


(0, 133, 0)

In [197]:
float_report(32)

Sign: 0, exp: 132, mant: 0x0


(0, 132, 0)

In [195]:
float_report(0.09375)

Sign: 0, exp: 123, mant: 0x400000


(0, 123, 4194304)

In [196]:
1.5 * 2**-4

0.09375

We now mask the 23 bits that make up the mantissa to set it to 0 so we don't need to worry too much about how the shifting rolls into the mantissa. This mask is given by 0xff80000, i.e. we let through the first 9 bits and block the next 23 - this can also be done via string construction

In [176]:
binarymask = "1" * 9 + "0" * 23
hex(int(binarymask,2))

'0xff800000'

In [198]:
def inv_sqrt(num):
    im = 0x5f000000 - ((cast_to_int(num) >> 1) & 0xff800000)
    return cast_to_float(im)

inv_sqrt(64)

0.125

In [199]:
inv_sqrt(32)

0.125

In [184]:
inv_sqrt(16)

0.25

In [207]:
inv_sqrt(1/64)

8.0

In [208]:
inv_sqrt(1/32)

4.0

What about the mantissa? 